# Data Analytics Fall 2025 &mdash; Exercises 1

### XXXXX XXXXX XXXXX XXXXX (last modified: Sun 31 Aug)

### Deadline: Around Tue 16 Sep (to be specified)

- Five problems
- Minor variations between users
- Theme: Python & Numpy (no Pandas allowed)
- Theory: see <tt>public/exrc_01</tt>
- Make a copy of the original notebook (e.g. <tt>File $\rightarrow$ Save Notebook As</tt>)<br/>
  and add your answers (new cells) there
- Please make both your code and your notebook readable
- Keep your folder structure up to date by running the config script:

In [1]:
import os
os.system('/usr/bin/bash /home/varpha/dan/config.sh');


Configuring...

  created the ~/dan directory tree
  changed all ~/dan subdir permissions to 700
  removed any broken filelinks under ~/dan
  copied filelinks from /home/varpha/dan to ~/dan
  removed any python cache dirs
  creating answers workbook (Darren's idea)
  answers workbook /home/XXXXX/dan/private/exrc_01/exrc_01_answers.ipynb already exists, skipping copy

  upgrading jupyterlab etc. (may take a while)...


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  done (you may need to restart your server in order for the upgrades to take effect)

All Done!

Please run this config script whenever you start working on the hub.

If you encountered errors, please re-run the script. If the errors persist, please report to our Teams channel.

Also, please do 'File -> Hub Control Panel -> Stop My Server'
whenever you stop working on the hub.

Thank you!



## Problem 1. Documentation
- Browse through the Python and Numpy documentation
- Find a function that a) interests you, and b) has a messy documentation
- Play with the function and find simple use cases
- Explain the function to your anonymous peer reviewer.

Please write a nice and clear explanation. Include some elementary examples.

I used ChatGPT for help me in this exercise.

The function I selected is numpy.linspace. It is the first time I am working with Python and the documentation for that function is difficult for me to understand, but that function creates vectors which can be useful.

The function is called in this form. numpy.linspace(start, stop, num)
-Start is the first number of the vector
-Stop is the last number of the vector
-Num is the number in the vector. Example: Num=4 the vector will have 4 number, first and last incluid.

So the function will be called. V1=numpy.linspace(1, 4, 4) and return V1=[1. 2. 3. 4.]

Examples of a moment of needed this function:
- You need to divide a line in same parts.
- You need calculate the velocity in diferent moments but with same seconds betwen for calculated de aceleration.
- You need no draw a soft curve and request a lot of point.

## Problem 2. Map, Lambda, Groupby
In this problem, only plain python may be used, no numpy.<br/>
The following links may be helpful:
- [sorting howto](https://docs.python.org/3/howto/sorting.html)
- [lambda sorting](https://blogboard.io/blog/knowledge/python-sorted-lambda)
- [itertools groupby](https://stackoverflow.com/questions/773/how-do-i-use-itertools-groupby).

Using the code cell below, read a csv (real wind turbine data) into a list of dicts.<br/>
Then do the following:
- a) using map, convert the timestamps into the format <b>MM/dd/yyyy HH:mm:ss</b>, e.g. 11/04/2018 09:10:43
- b) using sorted and lambda, sort the rows according to increasing rotorspeed
- c) add a column called <b><i>WindSpeed_Group</i></b> that contains the letter A, B or C, where A = less than 5mps, B = 5-10mps, C = more than 10mps. Try to use [itertools.groupby](https://docs.python.org/3/library/itertools.html#itertools.groupby) (although it may not be very smart).

In your handin, include the code that does a) - c) above. No need to save the modified data. Here is the code for reading the raw data:

In [1]:
from getpass import getuser
import csv
user = getuser()
csv_location = f'/home/varpha/dan/private/{user}' + \
                f'/exrc_01/data/prob2_{user}.csv'
csv_location = "prob2_ah4323.csv"
with open(csv_location) as handle:
    mydata = list(csv.DictReader(handle))

In [2]:
# a) I will use IA for help me with the code
#First we will look for the dates. This code displays the data from the file.
print(mydata[0])

for row in mydata[:5]:
    print(row["TimeStamp"])

{'TimeStamp': '2018-04-01 00:00:52.584', 'ReactivePower_kVAr': '907', 'Power_kW': '-1', 'WindSpeed_mps': '8.358698', 'ErrorCode': '416', 'GenState': '1', 'RunState': '3', 'RotorSpeed_rpm': '0.03000221', 'RotorSpeedAve': '0.001997522', 'RotorOverSpeed': '0', 'RotorUnderSpeed': '0'}
2018-04-01 00:00:52.584
2018-04-01 00:04:09.477
2018-04-01 00:07:54.942
2018-04-01 00:08:04.105
2018-04-01 00:08:11.285


In [3]:
#Now we know how are the dates, and can work with they
from datetime import datetime

def convert_timestamp(row):
    ts = row["TimeStamp"] # Read the original dates
    dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S.%f") # Save the original dates
    row["TimeStamp"] = dt.strftime("%m/%d/%Y %H:%M:%S")     # Save the dates in the new format
    return row

# Do the same with all dates.
mydata = list(map(convert_timestamp, mydata))

# Display the first fives dates to chek
for row in mydata[:5]:
    print(row["TimeStamp"])

04/01/2018 00:00:52
04/01/2018 00:04:09
04/01/2018 00:07:54
04/01/2018 00:08:04
04/01/2018 00:08:11


In [5]:
# b)
mydata_sorted = sorted(
    mydata,
    key=lambda row: float(row["RotorSpeed_rpm"]) if row["RotorSpeed_rpm"].strip() != "" else 0.0 #Empy cells is trated like 0
)
for row in mydata_sorted[:5]: #Displace some of the cells
    print(row["RotorSpeed_rpm"], row["TimeStamp"])

-0.2550188 04/24/2018 07:07:10
-0.2100155 04/26/2018 14:55:32
-0.1950144 04/10/2018 13:01:48
-0.1950144 04/23/2018 13:29:11
-0.1950144 04/23/2018 21:11:47


In [6]:
# c)
from itertools import groupby

for row in mydata_sorted: # Convert WindSpeed_mps to float if is string; If is empty, assig 0 to value
    ws = row["WindSpeed_mps"]
    if isinstance(ws, str):
        ws = ws.strip()
        row["WindSpeed_mps"] = float(ws) if ws != "" else 0.0
    else:
        row["WindSpeed_mps"] = float(ws)  # Is is float, maintains the type

mydata_sorted_ws = sorted(mydata_sorted, key=lambda row: row["WindSpeed_mps"]) # Sort by WindSpeed_mps to use groupby

def wind_group(speed): # Function to assign group
    if speed < 5:
        return "A"
    elif speed <= 10:
        return "B"
    else:
        return "C"

for key, group in groupby(mydata_sorted_ws, key=lambda row: wind_group(row["WindSpeed_mps"])): # Use groupby to assign WindSpeed_Group
    for row in group:
        row["WindSpeed_Group"] = key

for row in mydata_sorted_ws[:10]: # Show some rows
    print(row["WindSpeed_mps"], row["WindSpeed_Group"], row["TimeStamp"])



0.0 A 04/10/2018 12:15:11
0.0 A 04/10/2018 12:17:39
0.0 A 04/10/2018 12:19:11
0.0 A 04/10/2018 12:21:10
0.0 A 04/10/2018 12:21:16
0.0 A 04/10/2018 12:22:05
0.0 A 04/10/2018 12:23:23
0.0 A 04/10/2018 12:24:16
0.0 A 04/10/2018 12:29:32
0.0 A 04/10/2018 12:29:52


## Problem 3. Vectorization
- Some [general info](https://www.askpython.com/python-modules/numpy/vectorization-numpy)
- The code in <tt>dan/public/exrc_01/integrator.py</tt> contains rudimentary code,<br/>
  written in plain python, that numerically integrates a (math) function<br/>
  $f\colon \mathbb{R} \to \mathbb{R}$ over an interval $[a,b]$.
- Rewrite the code using numpy and vectorization.
- Introduce timings to measure the gain of vectorization.
- Use the (math) function $f(x)=- 14 x^{12} - 9 x^{11} - 8 x^{5} + 3$ and interval $[a,b] = [-9, 18]$ to test the code.
- Increase the number of subintervals in order to obtain a noticeable difference in the timings.

In your handin, include the rewritten code along with the timing measures.

In [7]:
# I will use IA to help me in this exercise

import numpy as np
import time


# Function to integrate
def f(x):
    return -14*x**12 - 9*x**11 - 8*x**5 + 3


# Vectorized version
def create_mesh_np(a, b, n):
    return np.linspace(a, b, n, endpoint=False)

def integrate_np(f, a, b, n):
    mesh_width = (b - a) / n
    left_endpoints = create_mesh_np(a, b, n)
    midpoints = left_endpoints + mesh_width / 2
    heights = f(midpoints)
    return np.sum(heights * mesh_width)


# Original version
def create_mesh(a, b, n):
    return [a+i*(b-a)/n for i in range(n)]

def integrate(f, a, b, n):
    sum_of_rectangles = 0
    left_endpoints = create_mesh(a,b,n)
    mesh_width = (b-a)/n
    for left_endpoint in left_endpoints:
        midpoint = left_endpoint + mesh_width/2
        height = f(midpoint)
        sum_of_rectangles += height * mesh_width
    return sum_of_rectangles


# Timings to measure
if __name__ == "__main__":
    a, b = -9, 18
    
    for n in [10_000, 100_000, 1_000_000]:
        print(f"\nNumber of subintervals: {n}")
        
        # Pure Python version
        start = time.time()
        result_py = integrate(f, a, b, n)
        end = time.time()
        print(f"Pure Python: {result_py:.6e}, time = {end - start:.6f} s")
        
        # Vectorized NumPy version
        start = time.time()
        result_np = integrate_np(f, a, b, n)
        end = time.time()
        print(f"NumPy version.: {result_np:.6e}, time = {end - start:.6f} s")


Number of subintervals: 10000
Pure Python: -2.329488e+16, time = 0.002309 s
NumPy version.: -2.329488e+16, time = 0.000342 s

Number of subintervals: 100000
Pure Python: -2.329488e+16, time = 0.020983 s
NumPy version.: -2.329488e+16, time = 0.003215 s

Number of subintervals: 1000000
Pure Python: -2.329488e+16, time = 0.176740 s
NumPy version.: -2.329488e+16, time = 0.016816 s


## Problem 4. Numpy arrays

- The directory <tt>dan/private/exrc_01/data</tt><br/>
  contains a csv file (<tt>prob4_XXXXX.csv</tt>) with some weather data.
- a) Use [numpy.genfromtxt](https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html) to read the file into a 2-dimensional numpy array.<br/>
  Use dtype=str in order to not lose the headers.
- b) Use Boolean masking to drop the rows that contain <tt>nan</tt> entries.
- c) Convert the time entries (standard timestamp) into a human-readable format of your choice.
- d) Add a new row that contains the averages of the columns, except <tt>nan</tt> for the time column.

In your handin, include the code that does a) - d) above. Do not include any saved data.

In [5]:
import numpy as np
from datetime import datetime
from getpass import getuser

user = getuser()
file_path = f"/home/varpha/dan/private/{user}/exrc_01/data/prob4_{user}.csv"
file_path = "prob4_ah4323.csv"
# a) Read the header and separate the body
data = np.genfromtxt(file_path, delimiter=",", dtype=str)
headers = data[0, :]
rows = data[1:, :]

# b) Eliminate the rows with nan entries
lower_rows = np.char.lower(rows)
is_nan = np.isin(lower_rows, ['nan'])
mask = ~np.any(is_nan, axis=1)
clean_rows = rows[mask]

# c) Convert the time into a human-readable format
timestamps = clean_rows[:, 3].astype(float).astype(int)
readable_times = [datetime.utcfromtimestamp(int(ts)).strftime("%Y-%m-%d %H:%M:%S")
                  for ts in timestamps]
clean_rows[:, 3] = readable_times

# d) Calculate averages for numeric columns (except time)
ncols = clean_rows.shape[1]
numeric_cols = [i for i in range(ncols) if i != 3]
numeric_data = clean_rows[:, numeric_cols].astype(float)
col_means = np.mean(numeric_data, axis=0)
avg_row = np.array(['nan'] * ncols, dtype=object)
for idx, mean in zip(numeric_cols, col_means):
    avg_row[idx] = mean
clean_rows_obj = clean_rows.astype(object)
final_data = np.vstack([clean_rows_obj, avg_row])

# Show results
print("Headers:")
print(headers)
print("\nClean data with legible times (includes row of averages at the end):")
print(final_data)
np.savetxt("output_formatted_data_averages.csv", final_data, delimiter=",", fmt="%s")


Headers:
['air_temperature_2m' 'relative_humidity_2m' 'snowfall_amount_acc' 'time'
 'x_wind_10m' 'y_wind_10m']

Clean data with legible times (includes row of averages at the end):
[['268.424561' '0.868437' '0.000000' '2023-01-04 00:00:00' '0.300850'
  '-2.382514']
 ['267.929047' '0.869720' '0.039103' '2023-01-04 01:00:00' '0.152142'
  '-2.623582']
 ['266.381226' '0.879852' '0.112630' '2023-01-04 04:00:00' '0.680576'
  '-2.917674']
 ...
 ['270.487732' '0.877308' '0.000651' '2023-01-02 22:00:00' '-1.110839'
  '-0.406309']
 ['270.797119' '0.862045' '0.013021' '2023-01-02 23:00:00' '-1.196100'
  '-0.412851']
 [np.float64(264.8180316666667) np.float64(0.8374361777777777)
  np.float64(0.15797621111111113) 'nan' np.float64(0.35346621111111115)
  np.float64(-0.2847849277777778)]]


/var/folders/0t/k860jmxx4d5__9h3p08nh77m0000gn/T/ipykernel_22460/1315020360.py:21: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  readable_times = [datetime.utcfromtimestamp(int(ts)).strftime("%Y-%m-%d %H:%M:%S")


## Problem 5. Data download
- Start by exploring / running the code in <tt>dan/public/exrc_01/statfi.py</tt>
- Choose a topic that interests you. Then try to download a "lot" of data of data of that topic. Here a lot means something like 500kB - 2MB range. (It's not really a lot but enough that the downloaded data is hard to grasp manually.)
- Save your data in one or several json files.

In your handin, include the code that you used (no saved data).
Also, tell a few words about your experiences. What problems, if any, did you encounter?

This is so dificult, I never use Python before and this exercise have a lot of problem u needed help of IA no fix my code.

In [40]:
##### imports #####

import requests
import json
import sys

# this has to do with pass by value / reference
from copy import deepcopy

##### config #####

english = True
# english = False


##### helpers #####


# notebook replacement of sys.exit()
# call with raise StopExecution
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

query_template = {
    "query": [], # list of query items
    "response": {
        "format": "json"
    }
}

query_item_template = {
    "code": "", # variable
    "selection": {
        "filter": "item",
        "values": [] # list of strings
    }
}


##### main #####


with requests.Session() as session:

    '''
    first, some browsing in order to get the correct database
    you can do this with a browser too (but translation may become an issue)
    '''

    lang_id = 'en' if english else 'fi'
    base_url = f'https://pxdata.stat.fi/PXWeb/api/v1/{lang_id}/StatFin'
    response = session.get(base_url)

    for item in response.json():
        print(item['id'], item['text'])

    # stop execution
    raise StopExecution

    '''
    next, append the id of your thing of interest to the url
    (EDIT the adopt below)
    '''

    catalogue_url = f'{base_url}/adopt'
    response = session.get(catalogue_url)

    '''
    check what .px files are available in the "catalogue"
    '''
    for item in response.json():
        print(item['id'], item['text'])

    # stop execution
    raise StopExecution

    '''
    once you decide what .px file interests you, 
    EDIT it below in order to fetch the available data headers

    '''

    headers_url = f'{base_url}/adopt/statfin_adopt_pxt_11lv.px'
    response = session.get(headers_url)

    myjson = response.json()
    print()
    print('variables:', len(myjson['variables']))
    print()
    for var in myjson['variables']:
        print(var['text'])
    print()

    if english:
        tmp_url = headers_url.replace('/en/','/fi/')
        response = session.get(tmp_url)
        myjson = response.json()
        print()
        print('the corresponding variables in finnish (may needed in the actual query):')
        print()
        for var in myjson['variables']:
            print(var['text'])
        print()

    # stop execution
    raise StopExecution

    '''
    okay, but then things get more serious as we build the actual query for the data

    first, fetch the maximum values that one can download
    (this is kind of hi-tech, got it from the documentation)
    (which typically sucks in free & public apis like this)
    '''
    response = session.get(f'https://statfin.stat.fi/PXWeb/api/v1/{lang_id}/?config')
    maxvalues = response.json()['maxValues']

    '''
    query building (we don't request anything yet)
    please edit only the "for myvar" line
    '''
    query = deepcopy(query_template)
    total_values = 1
    for myvar in ['Vuosi', 'Syntymävaltio', 'Ikä', 'Sukupuoli']: # EDIT this line
        myvalues = []
        query_item = deepcopy(query_item_template)
        for v in myjson['variables']:
            if v['code'] == myvar:
                myvalues = v['values']
        total_values = total_values * len(myvalues)
        query_item['code'] = myvar
        query_item['selection']['values'] = myvalues
        query['query'].append(query_item)
    if total_values > maxvalues:
        print('your query is too big, try again with fewer variables')
        raise StopExecution


    '''
    obtain the actual data with a "post" request
    that's like submitting a web form
    and cannot be done by gui browsing anymore
    '''
    response = session.post(headers_url, json=query)

    '''
    finally, dump the data to a file
    '''
    myjson = response.json()
    with open('test.json', 'w') as handle:
        json.dump(myjson, handle, indent=4)



matk Accommodation statistics
adopt Adoptions
tilma Air emission accounts
ilma Air transport
vtp Annual national accounts
mata Balance of payments and international investment position
kony Bankruptcies and business restructuring proceedings
synt Births
ras Building and dwelling production
rki Building cost index
raku Building stock and new production
rakke Buildings and free-time residences
bdem Business demography
ksyyt Causes of death
vkp Central government monthly salaries
ssaaty Changes in marital status
kans Citizenships granted
kbar Consumer confidence
khi Consumer price index
kivih Consumption of hard coal
maku Cost index of civil engineering works
alvaa County elections
klt Culture
klts Culture satellite accounts
cvts CVTS, continuing vocational training
kuol Deaths
uloa Debtors in enforcement
kkesk Discontinuation of education
kvliik Domestic waterborne traffic
asas Dwellings and housing conditions
vaka Early childhood education and care
eaa Economic accounts for agriculture 

In [8]:
import requests
import json
from copy import deepcopy

##### CONFIG #####
english = True  
base_url = f'https://pxdata.stat.fi/PXWeb/api/v1/{"en" if english else "fi"}/StatFin'

##### TEMPLATES #####
query_template = {
    "query": [],
    "response": {"format": "json"}
}

query_item_template = {
    "code": "",
    "selection": {
        "filter": "item",
        "values": []
    }
}

##### STEP 1: VIEW THE DATA SETS #####
with requests.Session() as session:
    catalogue_url = f"{base_url}/ava"
    response = session.get(catalogue_url)
    datasets = response.json()

    print("=== Topics in 'Subject choices of students'")
    for d in datasets:
        print(d["id"], "-", d["text"])

=== Topics in 'Subject choices of students'
statfin_ava_pxt_12a9.px - 12a9 -- Language choices of completers of grades 1 to 9 and of additional education of comprehensive education, school year 2020/2021 -- school year 2023/2024
statfin_ava_pxt_12aa.px - 12aa -- Language choices of students who have completed the curriculum in basic education for adults, 2020-2023
statfin_ava_pxt_12ad.px - 12ad -- Foreign languages selected by upper secondary level students, 2020-2023
statfin_ava_pxt_139d.px - 139d -- Number of foreign languages selected by upper secondary level students, 2020-2023
statfin_ava_pxt_139p.px - 139p -- Shares of number of foreign languages selected by upper secondary level students, 2020-2023
statfin_ava_pxt_14cd.px - 14cd -- Number of languages completed in grades 1 to 9 and in additional education in comprehensive education, school year 2020/2021 -- school year 2023/2024


In [9]:
import requests
import json

base_url = f'https://pxdata.stat.fi/PXWeb/api/v1/{"en" if english else "fi"}/StatFin'
dataset_id = "statfin_ava_pxt_12ad.px"

with requests.Session() as session:
    headers_url = f"{base_url}/ava/{dataset_id}"
    response = session.get(headers_url)
    myjson = response.json()

    print(f"=== Variables available in {dataset_id} ===")
    for var in myjson["variables"]:
        print(f"{var['code']} - {var['text']} ({len(var['values'])} valores)")

    fi_headers_url = headers_url.replace("/en/", "/fi/")
    response = session.get(fi_headers_url)
    myjson_fi = response.json()

    print("\n=== Variables in finish ===")
    for var in myjson_fi["variables"]:
        print(f"{var['code']} - {var['text']} ({len(var['values'])} valores)")


=== Variables available in statfin_ava_pxt_12ad.px ===
Vuosi - Year (4 valores)
Alue - Area (21 valores)
Sukupuoli - Gender (4 valores)
Koulutusaste - Level of education (3 valores)
Tiedot - Information (25 valores)

=== Variables in finish ===
Vuosi - Vuosi (4 valores)
Alue - Alue (21 valores)
Sukupuoli - Sukupuoli (4 valores)
Koulutusaste - Koulutusaste (3 valores)
Tiedot - Tiedot (25 valores)


In [10]:
from copy import deepcopy

query_template = {
    "query": [],
    "response": {"format": "json"}
}

query_item_template = {
    "code": "",
    "selection": {
        "filter": "item",
        "values": []
    }
}

with requests.Session() as session:
    headers_url = f"{base_url}/ava/{dataset_id}"
    fi_headers_url = headers_url.replace("/en/", "/fi/")
    response = session.get(fi_headers_url)
    myjson_fi = response.json()

    query = deepcopy(query_template)   # We build the query using all available values
    for v in myjson_fi["variables"]:
        query_item = deepcopy(query_item_template)
        query_item["code"] = v["code"]
        query_item["selection"]["values"] = v["values"]  # selec all
        query["query"].append(query_item)

    response = session.post(headers_url, json=query) # We download the data
    data = response.json()

    # Save the document
    with open("subject_choices_12ad.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print("✅ Data downloaded and saved in 'subject_choices_12ad.json''")



✅ Data downloaded and saved in 'subject_choices_12ad.json''


## How to submit my solutions?

Open a Terminal tab (e.g. <tt>File $\rightarrow$ New $\rightarrow$ Terminal</tt>, copy-paste the following into the Terminal command prompt, and press enter:
<pre>
  /home/varpha/dan/menu.py
</pre>